In [ ]:
# modules required; if not present; pip install
import pandas as pd
from calendar import monthrange
from selenium import webdriver
from selenium.webdriver.common.by import By 
import time
import re

In [ ]:
###Info which can be modified
year=2023 # selected year (can change)
start_mth=2 # where number refers to the month of the year; 1st mth being January; 12th month being DecembeR (any integer from 1-12 (inclusive))
end_mth=12 #(any integer greater than/ equal to the start_mth (cannot be more than 12))
budget=500 # Max cost of flight from SG to destination (2 way) (any realistic integer)
# can't really expect much results for $1 flight trips can you
###

base_url='https://www.kayak.sg/explore/SIN-anywhere/' # the base url used to check available destinations from SG

month=[]
all_costs=[]
all_places=[]

for m in range(start_mth,end_mth+1):
    last_day=str(monthrange(year, m)[-1])
    m=str(m)
    if len(m)==1:
        m='0'+m # where if month has only 1 digit; like for Aug (8); '0' is added In front of the character >> '08' 
        # (allows url to be valid and therefore can be scraped)
    
    start_yyyymmdd= f'2023{m}01'
    end_yyyymmdd=f'2023{m}{last_day}'
    search=f'{base_url}{start_yyyymmdd},{end_yyyymmdd}?budget={str(budget)}' #url with modifiers* is stringed together and assigned to variable
    # * >> includes modifiers into base url to show interested duration and budget
    # e.g. https://www.kayak.sg/explore/SIN-anywhere/20230301,20230331?budget=1000
    #   where all available destinations for the duration of the full month of January are searched
    #   and where the max flight cost for 2 way (SG to overseas airport) is 1000 SGD
    #   data is specific to kayaK
    
    dr=webdriver.Chrome() # selenium is used so as to allow for elements to be identified based on id/ className etc
    # as opposed to using bs4 
    dr.get(search)
    time.sleep(10) # allows webpage to load before scraping (10 seconds is a bit generous but time isn't that large of an issue personally)
    costs=(dr.find_elements(by=By.CLASS_NAME, value="Blek-title"))
    pattern='>(.*)<'

    zoom_out=dr.find_element(by=By.XPATH, value="//*[contains(@id,'minusButton')]")

    for e in range(0,4): # replicates this 4 times
        zoom_out.click() # zooms out on the map as to allow for a global overview on all possible destinations
        # if not done; will limit destinations to SEA only (aka what's shown on the map w/o zooming out)
        # zooming out on map also influences destinations sHown in the 'Explore Destinations' section of the page
        time.sleep(3) # allows webpage to load as some dynamic content may not be displayed immediately

    time.sleep(4) # allows webpage to load 1 final time for some dynamic content to be displayed
    button=dr.find_element(by=By.XPATH, value="//*[contains(@id,'showMoreButton')]") # selects the 'Load More Destinations' button 
    while True:
        try:
            button.click() 
            time.sleep(5) # code to click the load more destinations button as long as It is present; if no more destinations
        except: # except code is triggered (as button cannot be clicked; hence while loop is broken out of)
            break 
    # with all possible destinations loaded
    elements=(dr.find_elements(by=By.CLASS_NAME, value="_ihz._irp._iqB._ilc._iai"))
    for i in elements:
        e=(i.get_attribute('innerHTML'))
        if ';' not in e: # in htmL ';' is an indicator that the 'from S$ (cost)' tag is present the destination
            # necessary as some destinations do not have that tag (indicative that exceeds budget)
            # so only data with 'from S$ (cost)' tag is scraped; with name of place and cost being scraped
            break
        else:
            place,cost=re.findall(pattern,e)
            cost=(cost.split(';')[-1])
            all_places.append(place) # adds place to list
            all_costs.append(cost) # adds cost of gg to place to a separate list
            month.append(m) # adds month of travel to a different list


In [ ]:
month_year=[]
for m in month:
    month_year.append(m+'/'+str(year)) # from 02 to 02/2023 (based on existing year)

all_costs=[int(i) for i in all_costs] # converts type of value in all_costs list from string to integer
# values in HTML (being text) exist as strings

df=pd.DataFrame({'month':month_year,'place':all_places,'cost':all_costs}) # creates the dataframe

df.index=range(1,len(df)+1) # makes the start index 1 (instead of 0) > for better readability and to easily show number of locations scraped

df.to_excel(r'C:\Users\acer\OneDrive\Desktop'+r'\locations.xlsx') 
# exports df to excel sheet on desktop (will need to be modified based on computer)

#Exported Data can be Visualised in PowerBI using ArcGIS